# IMPORTS

In [ ]:
import torch
import pandas as pd
import torch.nn.functional as F
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# Mounting Gdrive

# Dataframe

In [ ]:
df=pd.read_csv('./paddy-dataset/train.csv')

# Checking Runtime Env

In [ ]:
device = 'cuda'if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Preprocessing the Data

In [ ]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(True),
    transforms.RandomVerticalFlip(True),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
    transforms.RandomAffine(degrees=5, shear=0.3, scale=(0.3, 0.7), translate=(0.05, 0.05)),
])


# Loading the Dataset

spliting the dataset

In [ ]:
import splitfolders as sf
path='./paddy-dataset/train_images'
sf.ratio(path,output='./splitted',ratio=(.8,.2,0),seed=1337)

Preparing the dataset for loading into the DataLoader

In [ ]:
training_ds=ImageFolder('./splitted/train/',transform=transform)
testing_ds=ImageFolder('./splitted/val/',transform=transform)

In [ ]:
train_loader = DataLoader(training_ds, batch_size=16, shuffle=True)
test_loader = DataLoader(testing_ds, batch_size=16, shuffle=False)
print(len(train_loader),len(test_loader))

521 130


Importing the Model

In [ ]:
from torchvision.models import vgg16
model = vgg16(pretrained=True)

d:\PYthon\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\PYthon\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MOdifying the last linear layer of the model based on the number classes

Preparing for training the Model

In [ ]:
import torch.nn as nn
num_classes = len(df['label'].unique())
model.classifier[-1] = nn.Linear(in_features=model.classifier[-1].in_features, out_features=num_classes)

In [ ]:
import torch.optim as optim
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)

training the model

In [ ]:
for epoch in range(20):  # loop over the dataset multiple times
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # inputs=inputs.view(-1,224*224*3)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print('epoch=',epoch,'completed')
print('Finished Training')

epoch= 0 completed
epoch= 1 completed
epoch= 2 completed
epoch= 3 completed
epoch= 4 completed
epoch= 5 completed
epoch= 6 completed


testing the model's Accuracy

In [ ]:
correct = 0
total = 0
predictions=[]
model.eval()
for i,data in enumerate(test_loader,0):
        images, labels = data
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        predictions.append(outputs)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the  test images: {100 * correct // total} %')

Saved the trained parameters for later use

In [ ]:
torch.save(model.state_dict(),f='mobilenet_v2_paddy_states')